In [70]:
import pandas as pd

In [1]:
articles = pd.read_csv(
    '../data/articles_prep.csv', 
    sep=';',
    index_col=['company', 'time_stamp']
).sort_index()

stock_prices = pd.read_csv(
    '../data/stocks_prices_prep.csv', 
    sep=';',
    index_col=['company', 'time_stamp']
)

index = pd.read_csv(
    '../data/index_prep.csv', 
    sep=';',
    index_col=['index', 'time_stamp']
)

index = (index.loc['^MDAXI', :] + index.loc['^SDAXI', :]) / 2
index = index.rename(columns={
    'price': 'index'
})

stock_prices = stock_prices.merge(
    index,
    right_index=True,
    left_index=True,
)

stock_prices['alpha'] = stock_prices['price'] - stock_prices['index']

display(articles)
display(stock_prices)

NameError: name 'pd' is not defined

In [76]:
stock_prices['alpha_3'] = stock_prices.groupby(['company'])['alpha'].transform(lambda x: x.rolling(3).sum().shift(-2))
#stock_prices['alpha_5'] = stock_prices.groupby(['company'])['alpha'].transform(lambda x: x.rolling(5).sum().shift(-4))
stock_prices.loc[stock_prices['alpha_3'].between(-0.005, 0.005), 'movement'] = 'same'
stock_prices.loc[stock_prices['alpha_3'] > 0.005, 'movement'] = 'up'
stock_prices.loc[stock_prices['alpha_3'] < -0.005, 'movement'] = 'down'
display(stock_prices)

price     index     alpha   alpha_3 movement
company          time_stamp                                                 
1&1 drillisch ag 2018-01-01  0.000000  0.000000  0.000000 -0.001496     same
                 2018-01-02  0.021255  0.000181  0.021074 -0.005771     down
                 2018-01-03 -0.011519  0.011051 -0.022570 -0.029912     down
                 2018-01-04  0.006496  0.010771 -0.004275  0.005932       up
                 2018-01-05  0.005739  0.008805 -0.003067  0.028595       up
...                               ...       ...       ...       ...      ...
zooplus ag       2022-09-05       NaN -0.020870       NaN       NaN      NaN
                 2022-09-06       NaN  0.005961       NaN       NaN      NaN
                 2022-09-07       NaN  0.003477       NaN       NaN      NaN
                 2022-09-08       NaN  0.005395       NaN       NaN      NaN
                 2022-09-09       NaN  0.020202       NaN       NaN      NaN

[139650 rows x 5 columns]

In [80]:
tf_dataset = articles.merge(
    stock_prices,
    right_index=True,
    left_index=True, 
    how='left'
)

tf_dataset = tf_dataset[[
    'title',
    'text',
    'movement',
]]

tf_dataset.to_csv('../data/tf_dataset.csv', sep=';')
display(tf_dataset)

title  \
company          time_stamp                                                      
1&1 drillisch ag 2018-01-12  EQS-News: Drillisch AG: Drillisch Turns into '...   
                 2018-03-21  DGAP-Adhoc: 1&1 Drillisch AG on a Steady Cours...   
                 2018-05-09  EQS-News: 1&1 Drillisch Aktiengesellschaft: Su...   
                 2018-05-09  1&1 Drillisch AG Q1 EPS Down, EBIT Climbs; Con...   
                 2018-05-09  DGAP-News: 1&1 Drillisch Aktiengesellschaft: S...   
...                                                                        ...   
zooplus ag       2021-11-08  DGAP-Adhoc: zooplus AG: Decision by Zorro Bidc...   
                 2021-11-16  DGAP-News: zooplus AG: robust sales growth con...   
                 2021-11-17  EQS-PVR: zooplus AG: Correction of a release f...   
                 2021-11-22  EQS-PVR: zooplus AG: Release according to Arti...   
                 2021-12-07  DGAP-PVR: zooplus AG: Release according to Art...   

                                                                          text  \
company          time_stamp                                                      
1&1 drillisch ag 2018-01-12  DGAP-News: Drillisch AG / Key word(s): AGM/EGM...   
                 2018-03-21  1&1 Drillisch AG on a Steady Course of Growth ...   
                 2018-05-09  Forecast 2018 confirmed DGAP-News: 1&1 Drillis...   
                 2018-05-09  MAINTAL (dpa-AFX) - 1&1 Drillisch AG (DRHKF.PK...   
                 2018-05-09  1&1 Drillisch Aktiengesellschaft: Successful S...   
...                                                                        ...   
zooplus ag       2021-11-08  zooplus AG: Decision by Zorro Bidco S.\xE0 r.l...   
                 2021-11-16  zooplus AG: robust sales growth continues in t...   
                 2021-11-17  Article 40, Section 1 of the WpHG [the German ...   
                 2021-11-22  [the German Securities Trading Act] with the o...   
                 2021-12-07  DGAP Voting Rights Announcement: zooplus AG zo...   

                            movement  
company          time_stamp           
1&1 drillisch ag 2018-01-12     down  
                 2018-03-21     down  
                 2018-05-09       up  
                 2018-05-09       up  
                 2018-05-09       up  
...                              ...  
zooplus ag       2021-11-08     same  
                 2021-11-16       up  
                 2021-11-17       up  
                 2021-11-22       up  
                 2021-12-07     down  

[22394 rows x 3 columns]